In [4]:
import json
import os
from urllib.parse import urlencode, urljoin
from dotenv import load_dotenv
import pandas as pd
import requests

load_dotenv()

True

In [5]:
headers:dict = {
    'Ocp-Apim-Subscription-Key': os.getenv('API_KEY_DATOS_FINANCIEROS'), #Sustituir con tu API Key
    'User-Agent': (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/111.0.0.0 Safari/537.36"
    ),
}

base_url:str = 'https://apis.sb.gob.do/estadisticas/v2/'


In [6]:
def obtener_datos_financieros_rd(end_point: str, params:dict) -> pd.DataFrame:
    """
    Obtiene datos financieros desde una API REST y devuelve un DataFrame de pandas.

    Esta función realiza múltiples solicitudes a un endpoint de la API de 
    datos públicos de la Superintendencia de Bancos de la República Dominicana 
    para recuperar datos financieros, paginando a través de los resultados. 
    Los datos se recopilan en un DataFrame de pandas que se devuelve al finalizar.

    Parámetros
    ----------
    end_point : str
        El endpoint de la API que se utilizará para la solicitud de datos.
        
    params : str
        Filtros a ser aplicados en la petición al endpoint

    Returns
    -------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos financieros obtenidos 
        del API.

    Raises
    ------
    requests.HTTPError
        Si la respuesta de la API indica un error HTTP.
    
    json.JSONDecodeError
        Si la respuesta JSON no puede ser decodificada correctamente.

    Exception
        Si ocurre un error inesperado durante el proceso de obtención de datos.
    
    Ejemplo
    ------
    obtener_datos_financieros_rd('indicadores/morosidad-estresada','2024-01','2024-04')

    Notes
    -----
    - La función gestiona la paginación automáticamente, asegurándose de que 
      se recuperen todos los registros disponibles en el rango de fechas especificado.
    - Se asume que la respuesta de la API incluye un encabezado `x-pagination` 
      con metadatos que indican si hay más páginas de datos disponibles.
    - Es necesario tener la biblioteca `requests` y `pandas` instaladas en el entorno 
      donde se ejecute esta función.
    """
    next_page = True
    registros_list: list[pd.DataFrame] = []

    while next_page:
    
        # Construir la URL final
        url = urljoin(base_url, end_point) + '?' + urlencode(params)

        try:
            # Hacemos la petición
            response = requests.get(url, headers=headers)
            # El método raise_for_status() es una función de la biblioteca requests 
            # que se utiliza para verificar si la solicitud fue exitosa
            #Códigos de Éxito desde 200 hasta 299
            #Códigos de Error:  desde  Si el código de estado es 4xx o 5xx 
            #(por ejemplo, 404 Not Found, 500 Internal Server Error)
            response.raise_for_status()
            
            # Parseamos la respuesta con los datos a JSON
            datos = response.json()
            
            # Agregamos los datos a una lista conformada por DataFrames de pandas
            registros_list.append(pd.DataFrame(datos))
      
            # Obtenemos los metadatos necesarios de la respuesta
            metadatos = json.loads(response.headers['x-pagination'])
            next_page = metadatos['HasNext']
            total_pages = metadatos['TotalPages']
            total_record = metadatos["TotalRecords"]
            
            # Imprimimos la página actual y el total de páginas
            print('Página:', params['paginas'], '/', total_pages)
            
            # Incrementamos 1 para ir a la siguiente página
            params['paginas'] += 1
            
        except requests.HTTPError as http_err:
            print("Error en la petición HTTP:", http_err)
            raise
        except json.JSONDecodeError as json_err:
            print("Error al decodificar la respuesta JSON:", json_err)
            raise
        except Exception as e:
            print("Ha ocurrido un error:", e)
            raise

    # Concatenamos la lista de DataFrames en uno solo
    registros_finales = pd.concat(registros_list, ignore_index=True)

    return registros_finales

In [7]:
params = dict(
             periodoInicial = "2022-01", # Fecha Inicial
             periodoFinal = "2022-02",
             tipoEntidad = "BM",
             paginas = 1,
             registros = 50_000,
)

In [8]:
obtener_datos_financieros_rd('indicadores/financieros',params)

Página: 1 / 1


,periodo,tipoEntidad,entidad,indicador,tipoIndicador,valor,unidad
0,2022-01,BANCOS MÚLTIPLES,SANTA CRUZ,Gastos Generales y Administrativos / Margen Op...,Gestión,77.74,%
1,2022-01,BANCOS MÚLTIPLES,BANESCO,Número de Empleados / Número de Oficinas,Gestión,33.86,PERSONAS
2,2022-01,BANCOS MÚLTIPLES,BANRESERVAS,Número de Empleados / Número de Oficinas,Gestión,35.94,PERSONAS
3,2022-01,BANCOS MÚLTIPLES,CARIBE,Número de Empleados / Número de Oficinas,Gestión,40.45,PERSONAS
4,2022-01,BANCOS MÚLTIPLES,TODOS,Cobertura de Cartera de Crédito Vencida Mayor ...,Estructura de la cartera de créditos,365.29,%
...,...,...,...,...,...,...,...
2475,2022-02,BANCOS MÚLTIPLES,BLH,Cartera de Crédito Vencida (Capital ) / Total ...,Estructura de la cartera de créditos,2.40,%
2476,2022-02,BANCOS MÚLTIPLES,TODOS,Cartera de Crédito Vencida (Capital ) / Total ...,Estructura de la cartera de créditos,1.17,%
2477,2022-02,BANCOS MÚLTIPLES,TODOS,Margen de Intermediación Neto,Rentabilidad,6.60,%
2478,2022-02,BANCOS MÚLTIPLES,SANTA CRUZ,Margen de Intermediación Neto,Rentabilidad,5.64,%
